## 📘 Script 1 – Carregamento e diagnóstico inicial

Neste primeiro passo apenas carregamos os dados brutos e fazemos uma leitura geral.
Isso ajuda a entender a estrutura, ver problemas óbvios e planejar o tratamento.


In [11]:
import pandas as pd

# Caminhos dos arquivos originais
path_telemetria = r"C:\Users\anaca\OneDrive\Área de Trabalho\projetosenai\telemetria_detalhada_30dias.csv"
path_producao   = r"C:\Users\anaca\OneDrive\Área de Trabalho\projetosenai\historico_producao_1ano.csv"
path_eventos    = r"C:\Users\anaca\OneDrive\Área de Trabalho\projetosenai\eventos_industriais.csv"

# Leitura dos dados brutos
telemetria_df = pd.read_csv(path_telemetria)
producao_df   = pd.read_csv(path_producao)
eventos_df    = pd.read_csv(path_eventos)

print("TELEMETRIA")
display(telemetria_df.head())
display(telemetria_df.info())

print("\nPRODUÇÃO")
display(producao_df.head())
display(producao_df.info())

print("\nEVENTOS")
display(eventos_df.head())
display(eventos_df.info())

TELEMETRIA


,timestamp,maquina_id,ciclo_tempo_s,pressao_mpa,umidade_pct,temp_matriz_c,pecas_produzidas,flag_defeito
0,2025-10-20 10:54:19.143015,1,7.19,16.47,11.92,57.0,2,0
1,2025-10-20 10:54:19.143015,2,6.71,15.02,11.43,59.0,2,0
2,2025-10-20 10:54:26.143015,1,7.23,15.26,11.28,59.1,2,0
3,2025-10-20 10:54:26.143015,2,7.35,14.62,11.65,58.8,2,0
4,2025-10-20 10:54:33.143015,1,7.11,14.29,9.91,NaN,2,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648000 entries, 0 to 647999
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   timestamp         648000 non-null  object 
 1   maquina_id        648000 non-null  int64  
 2   ciclo_tempo_s     648000 non-null  float64
 3   pressao_mpa       648000 non-null  float64
 4   umidade_pct       648000 non-null  float64
 5   temp_matriz_c     643642 non-null  object 
 6   pecas_produzidas  648000 non-null  int64  
 7   flag_defeito      648000 non-null  int64  
dtypes: float64(3), int64(3), object(2)
memory usage: 39.6+ MB


None


PRODUÇÃO


...

None


EVENTOS


,timestamp,maquina_id,evento,severidade,origem
0,2025-05-07 07:54:19.143015,2,Parada Programada,Média,Operador
1,2025-05-24 23:54:19.143015,2,Parada Programada,Alta,Sensor
2,2025-10-16 05:54:19.143015,2,Parada Programada,Alta,Sensor
3,2025-04-16 16:54:19.143015,2,Parada Programada,Média,SCADA
4,2025-07-24 05:54:19.143015,1,Baixa Umidade,Alta,Sensor


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   timestamp   200 non-null    object
 1   maquina_id  200 non-null    int64 
 2   evento      200 non-null    object
 3   severidade  200 non-null    object
 4   origem      200 non-null    object
dtypes: int64(1), object(4)
memory usage: 7.9+ KB


None

## ⚙️ Script 2 – Tratamento da Telemetria (Silver)

Nesta etapa corrigimos a coluna de temperatura, arrumamos o timestamp
e deixamos a tabela pronta para análise. A ideia é apenas tornar os 
dados confiáveis, sem alterar a lógica.


In [12]:
import re

# Converter timestamp
telemetria_df["timestamp"] = pd.to_datetime(telemetria_df["timestamp"], errors="coerce")

# Função simples para limpar temperatura
def clean_temp(x):
    if pd.isna(x):
        return None
    x = re.sub(r"[^0-9\.,-]", "", str(x))  # remove letras e símbolos estranhos
    x = x.replace(",", ".")                # padroniza decimal
    try:
        return float(x)
    except:
        return None

telemetria_df["temp_matriz_c"] = telemetria_df["temp_matriz_c"].apply(clean_temp)

# Preenche falhas usando a última leitura válida
telemetria_df["temp_matriz_c"] = telemetria_df["temp_matriz_c"].fillna(method="ffill")

# Ordenar por máquina e tempo
telemetria_df = telemetria_df.sort_values(["maquina_id", "timestamp"]).reset_index(drop=True)

print("Telemetria tratada:")
display(telemetria_df.head())
telemetria_df.info()


Telemetria tratada:


C:\Users\anaca\AppData\Local\Temp\ipykernel_22736\3222045850.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  telemetria_df["temp_matriz_c"] = telemetria_df["temp_matriz_c"].fillna(method="ffill")


,timestamp,maquina_id,ciclo_tempo_s,pressao_mpa,umidade_pct,temp_matriz_c,pecas_produzidas,flag_defeito
0,2025-10-20 10:54:19.143015,1,7.19,16.47,11.92,57.0,2,0
1,2025-10-20 10:54:26.143015,1,7.23,15.26,11.28,59.1,2,0
2,2025-10-20 10:54:33.143015,1,7.11,14.29,9.91,58.8,2,1
3,2025-10-20 10:54:40.143015,1,7.09,14.22,12.09,55.0,2,1
4,2025-10-20 10:54:47.143015,1,7.46,15.34,11.32,60.0,2,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648000 entries, 0 to 647999
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   timestamp         648000 non-null  datetime64[ns]
 1   maquina_id        648000 non-null  int64         
 2   ciclo_tempo_s     648000 non-null  float64       
 3   pressao_mpa       648000 non-null  float64       
 4   umidade_pct       648000 non-null  float64       
 5   temp_matriz_c     648000 non-null  float64       
 6   pecas_produzidas  648000 non-null  int64         
 7   flag_defeito      648000 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 39.6 MB


## 🧱 Script 3 – Tratamento da Produção (Silver)

Aqui apenas criamos indicadores úteis, como taxa de refugo e se a 
máquina realmente produziu no período. Também ajustamos o timestamp 
e deixamos tudo organizado.

In [13]:
# Ajustar timestamp
producao_df["timestamp"] = pd.to_datetime(producao_df["timestamp"], errors="coerce")

# Taxa de refugo
producao_df["taxa_refugo"] = producao_df["pecas_refugadas"] / producao_df["pecas_produzidas"]

# Indica se houve produção
producao_df["produziu_algo"] = (producao_df["pecas_produzidas"] > 0).astype(int)

# Preencher eventuais NaN
producao_df["taxa_refugo"] = producao_df["taxa_refugo"].fillna(0)

# Organizar ordem
producao_df = producao_df.sort_values(["maquina_id", "timestamp"]).reset_index(drop=True)

print("Produção tratada:")
display(producao_df.head())
producao_df.info()


Produção tratada:


,timestamp,maquina_id,turno,pecas_produzidas,pecas_refugadas,consumo_kwh,status,taxa_refugo,produziu_algo
0,2024-11-19 10:54:19.143015,1,Manhã,937,16,9.709,Operando,0.017076,1
1,2024-11-19 11:54:19.143015,1,Manhã,956,20,10.151,Operando,0.020921,1
2,2024-11-19 12:54:19.143015,1,Manhã,928,11,9.931,Operando,0.011853,1
3,2024-11-19 13:54:19.143015,1,Manhã,905,15,9.057,Operando,0.016575,1
4,2024-11-19 14:54:19.143015,1,Tarde,928,21,9.965,Operando,0.022629,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17522 entries, 0 to 17521
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   timestamp         17522 non-null  datetime64[ns]
 1   maquina_id        17522 non-null  int64         
 2   turno             17522 non-null  object        
 3   pecas_produzidas  17522 non-null  int64         
 4   pecas_refugadas   17522 non-null  int64         
 5   consumo_kwh       17522 non-null  float64       
 6   status            17522 non-null  object        
 7   taxa_refugo       17522 non-null  float64       
 8   produziu_algo     17522 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(1), int64(3), object(2)
memory usage: 1.1+ MB


## 🚨 Script 4 – Tratamento dos Eventos (Silver)

Aqui deixamos os eventos mais consistentes, padronizando textos e 
criando um código de severidade que facilita análises futuras.

In [15]:
# Ajuste de timestamp
eventos_df["timestamp"] = pd.to_datetime(eventos_df["timestamp"], errors="coerce")

# Padroniza campos de texto para evitar variações bobas
eventos_df["evento"] = eventos_df["evento"].str.strip().str.title()
eventos_df["severidade"] = eventos_df["severidade"].str.strip().str.capitalize()
eventos_df["origem"] = eventos_df["origem"].str.strip().str.capitalize()

# Mapeamento simples para transformar texto em código numérico
map_sev = {"Baixa": 1, "Média": 2, "Alta": 3}
eventos_df["sev_codigo"] = eventos_df["severidade"].map(map_sev).fillna(0).astype(int)

# Ordena só para manter coerência temporal
eventos_df = eventos_df.sort_values(["maquina_id", "timestamp"]).reset_index(drop=True)

print("Eventos tratados:")
display(eventos_df.head())
eventos_df.info()

Eventos tratados:


,timestamp,maquina_id,evento,severidade,origem,sev_code,sev_codigo
0,2024-11-20 17:54:19.143015,1,Falha De Motor,Baixa,Sensor,1,1
1,2024-11-22 16:54:19.143015,1,Sobrecarga Elétrica,Alta,Scada,3,3
2,2024-11-27 03:54:19.143015,1,Sobrecarga Elétrica,Baixa,Sensor,1,1
3,2024-11-27 15:54:19.143015,1,Parada Programada,Média,Operador,2,2
4,2024-11-27 19:54:19.143015,1,Falha De Motor,Baixa,Operador,1,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   timestamp   200 non-null    datetime64[ns]
 1   maquina_id  200 non-null    int64         
 2   evento      200 non-null    object        
 3   severidade  200 non-null    object        
 4   origem      200 non-null    object        
 5   sev_code    200 non-null    int64         
 6   sev_codigo  200 non-null    int32         
dtypes: datetime64[ns](1), int32(1), int64(2), object(3)
memory usage: 10.3+ KB
